In [1]:
import glob
import pandas as pd
import os
# tested in transformers==4.18.0 
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import torch.nn.functional as F
import concurrent.futures


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Step 1: Download and save the tokenizer and model
def download_and_save_finbert():
    tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

    # Save locally
    tokenizer.save_pretrained("NLP/finbert-tone")
    model.save_pretrained("NLP/finbert-tone")
    print("Model and tokenizer saved locally.")

# Step 2: Load the tokenizer and model from local storage
def load_finbert_locally():
    tokenizer = BertTokenizer.from_pretrained("/NLP/finbert-tone")
    model = BertForSequenceClassification.from_pretrained("NLP/finbert-tone",num_labels=4)
    return tokenizer, model

#download_and_save_finbert()
# Load the model and tokenizer from local storage
tokenizer, model = load_finbert_locally()



In [ ]:
print(tokenizer)
print(model)

In [2]:

# Specify the folder path
folder_path = 'Test folder'
# Use glob to get all .csv files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))


In [3]:
def split_long_sentence(sentence, tokenizer, max_length=512):
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) <= max_length:
        return [sentence]
    
    # Split the tokens into chunks
    chunks = []
    for i in range(0, len(tokens), max_length):
        chunk_tokens = tokens[i:i + max_length]
        chunk_sentence = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunks.append(chunk_sentence)
        print("Run Chunk tokens process")
    
    return chunks

In [ ]:
appl2020 = pd.read_csv('AAPL-2010.csv')
for i, row in appl2020.iterrows():
        sen = row['Sentences']
        tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
        sentences = split_long_sentence(sen, tokenizer, 512)
        print(sentences)



In [4]:
def loadFinbertESG (df:pd):
    # Load the pre-trained model and tokenizer
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
    labels = ['None', 'Environmental', 'Social', 'Governance']
    for i,row in df.iterrows():
        sen = row['Sentences']
        sen = split_long_sentence(sen, tokenizer, 512)
        inputs = tokenizer(sen, return_tensors='pt',truncation=True, max_length=512, padding='max_length')
        outputs = finbert(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        results = {label: prob.item() for label, prob in zip(labels, probs[0])}
        df.loc[i,'Env'] = results['Environmental']
        df.loc[i,'Soc'] = results['Social']
        df.loc[i,'Gov'] = results['Governance']
        df.loc[i,'None'] = results['None']
        #print(f'{i}+:::{results}')
    return df

# showresult = loadFinbertESG(dfs)
# display(showresult)

In [ ]:
for file in csv_files:
    df = pd.read_csv(file)
    df = df.rename(columns={'0': 'Sentences'})
    print(f"{file}: {len(df)}")
    display(df)
    loadFinbertESG(df)
    processed_file_path = os.path.join(folder_path, os.path.basename(file))
    df.to_csv(processed_file_path, index=False)
    



In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())


In [5]:
def extract_details_from_filename(filename):
    parts = filename.replace('.csv', '').split('-')
    CompanyTicker = parts[0]
    Year = int(float(parts[1]))  # Convert to float first, then to int to handle any decimal points
    return CompanyTicker, Year

In [6]:
def process_file(file,Record):
    try:
    # 尝试使用默认设置读取CSV文件
        df = pd.read_csv(file)
        df = df.rename(columns={'0': 'Sentences'})
        #df = df.rename(columns={'Sentences': 'Sentences'})
        if len(df) >= 30:
            print(f"{file}: {len(df)}")
            # Assuming loadFinbertESG is defined elsewhere
            loadFinbertESG(df)
            processed_file_path = os.path.join(folder_path, os.path.basename(file))
            df.to_csv(processed_file_path, index=False)
            Record.loc[len(Record.index)]=[file,len(df),True]
        else:
            Record.loc[len(Record.index)]=[file,len(df),False]
            os.remove(file)
    except Exception as e:
        Record.loc[len(Record.index)]=[file,0,False]
        print(f"{file}读取文件时发生错误: {e}")
        os.remove(file)
    


In [7]:
if torch.backends.mps.is_available():
   mps_device = torch.device("mps")

In [8]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
# Use ThreadPoolExecutor to process files in parallel
FinBertRec = pd.DataFrame(columns=['Filepath','CSV.Length','Status'])
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_file, file,FinBertRec) for file in csv_files]
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"An error occurred: {e}")

display(FinBertRec)

/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/GLW-2010.csv: 1148
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/CRUS-2018.csv: 471
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/ON-2016.csv: 1131
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/TDY-2017.csv: 1377
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/SYNA-2018.csv: 697
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/CRUS-2024.csv: 750
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/MSI-2016.csv: 1863
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/VSAT-2016.csv: 1257
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/SLAB-2016.csv: 823
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/MXL-2018.csv: 1490
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/CSCO-2016.csv: 692
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/FFIV-2020.csv: 844
/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/TXN-2011.csv: 38
/Users/qjy/Desktop/FinTech/

/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:113

Run Chunk tokens process
Run Chunk tokens process


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/NEON-2019.csv: 658


...

/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/Users/qjy/Desktop/FinTech/Project/Test/New Report Data/UI-2017.csv: 1174


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process
Run Chunk tokens process


,Filepath,CSV.Length,Status
0,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,38,True
1,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,451,True
2,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,471,True
3,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,658,True
4,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,750,True
...,...,...,...
752,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,1157,True
753,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,1174,True
754,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,1156,True
755,/Users/qjy/Desktop/FinTech/Project/Test/New Re...,858,True
